In [ ]:
DATABRICKS_TOKEN = ''
DATABRICKS_HTTP_PATH = ""
DATABRICKS_SERVER_HOSTNAME =""
database = ""
HOST_HTTP_PATH = ""
SERVER_NAME =  ""
warehouse_id = ""

In [2]:
HOST = SERVER_NAME+'/'+HOST_HTTP_PATH

In [ ]:
from langchain.sql_database import SQLDatabase

db = SQLDatabase.from_databricks(catalog="", schema="", engine_args={"pool_pre_ping": True},host=HOST,api_token=DATABRICKS_TOKEN,warehouse_id="")

In [4]:
def extract_table_metadata(table_name):
    describe_output = db.run(f"DESCRIBE EXTENDED {table_name}")
    processed_output = eval(describe_output) if isinstance(describe_output, str) else describe_output

    column_details = []
    table_description = "No description available"

    for row in processed_output:
        if row[0].strip().lower() == "comment":  # Extract table description
            table_description = row[1]
        elif (
            len(row) == 3  # Ensure valid column format
            and row[0].strip()  # Ignore empty column names
            and row[0].strip() not in ["#", "", "Detailed Table Information", "Catalog", "Database", "Table", 
                                       "Created Time", "Last Access", "Created By", "Type", "Location", "Provider", 
                                       "Owner", "Is_managed_location", "Predictive Optimization", "Table Properties"]
        ):
            column_details.append({
                "name": row[0], 
                "type": row[1], 
                "description": row[2] if row[2] else "No description available"
            })

    return {
        "table_name": table_name,
        "description": table_description,
        "columns": column_details
    }


In [5]:
tables = db.get_usable_table_names()

In [6]:
table_metadata=[]
for table in tables:
    table_metadata.append(extract_table_metadata(table))

In [7]:
def clean_metadata(table_metadata):
    for table in table_metadata:
        table["columns"] = [
            col for col in table["columns"] if col["name"].strip() and col["name"] != "# Detailed Table Information"
        ]
    return table_metadata

clean_metadata = clean_metadata(table_metadata)


In [ ]:
from azure.identity import DefaultAzureCredential,ManagedIdentityCredential, get_bearer_token_provider
import openai
from langchain_openai import AzureChatOpenAI
from openai import AzureOpenAI

# Define the required scope
scope = 

# Initialize the credential
credential = ManagedIdentityCredential()

token_provider = get_bearer_token_provider(credential, scope)

# Ensure you request a token with the correct scope
token = credential.get_token(scope)

client = openai.AzureOpenAI(
    api_version="",
    azure_endpoint="",
    azure_ad_token_provider=token_provider
)

from langchain_openai import AzureOpenAIEmbeddings

embedding_model = AzureOpenAIEmbeddings(
    azure_deployment="",  # Set the correct Azure deployment name
    azure_endpoint="",
    api_version="",
    azure_ad_token_provider=token_provider
)

llm_instance = AzureChatOpenAI(
                azure_deployment="",
                api_version="",
                temperature=0,
                max_tokens=None,
                timeout=None,
                #seed = azure_config.get('seed'),
                max_retries=2,
                azure_endpoint="",
                azure_ad_token_provider=token_provider
            )

In [ ]:
def get_embeddings(text):
    token_provider = get_bearer_token_provider(
    DefaultAzureCredential(), ""
    )

    client = AzureOpenAI(
        api_version="",
        azure_endpoint="",
        azure_ad_token_provider=token_provider
    )
    response = client.embeddings.create(
        model="",
        input=text
    )
    return response.data[0].embedding

In [10]:
# Convert each table's metadata into a structured text format
table_texts = [
    f"Table: {table['table_name']}\nDescription: {table['description']}\nColumns: {table['columns']}"
    for table in clean_metadata
]

In [11]:
import numpy as np

embeddings = np.array([get_embeddings(text) for text in table_texts])

In [12]:
import faiss
# Step 3: Store embeddings in FAISS
dimension = embeddings.shape[1]  # Get embedding dimension
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

In [13]:
def query_faiss(query_text):
    query_embedding = np.array(get_embeddings(query_text)).reshape(1, -1).astype("float32")  # Format for FAISS

    # Perform FAISS search to find relevant tables
    k = 3  # Number of relevant tables to retrieve
    distances, indices = index.search(query_embedding, k)

    # Extract relevant table metadata
    relevant_table_metadata = [table_texts[i] for i in indices[0]]

    import ast

    # Process metadata in a structured way
    formatted_metadata = []
    for data in relevant_table_metadata:
        table_name = data.split('\n')[0].replace('Table: ', '')
        description = data.split('\n')[1].replace('Description: ', '')

        # Extract and parse column metadata
        column_data_raw = data.split('Columns: ')[1]  # Extract raw column string
        column_data = ast.literal_eval(column_data_raw)  # Convert string to Python list

        # Format columns with type and description
        columns = "\n".join([
            f"- {col['name']} ({col['type']}): {col['description'] if col['description'] else 'No description available'}"
            for col in column_data
        ])

        # Append formatted metadata
        formatted_metadata.append(f"**Table:** {table_name}\n**Description:** {description}\n**Columns:**\n{columns}")

    # Join all formatted metadata entries into a final structured text
    context_text = "\n\n".join(formatted_metadata)

    return context_text

In [14]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder


class PromptTemplate():
    def __init__(self) -> None:
        pass

    string_template = """
            You are a helpful AI assistant expert in querying SQL Databases to find answers to the user's questions. \
        Given an input question, first create a syntactically correct {dialect} SQL query to run, then look at the results of the query and provide a meaningful, human-readable answer to the input question. \
        Do not return the raw SQL query result directly as the final answer. Instead, interpret the result and explain it in a concise and clear manner. \
        You can order the results by a relevant column to return the most interesting examples in the database. \
        Unless otherwise specified, do not return more than {top_k} rows. Here is the relevant table info: {table_info}\
 
        ===Response Guidelines
        1. Follow these instructions for creating syntactically correct SQL queries: \
            - Be sure not to query for columns that do not exist in the tables and use alias only where required.\
            - Include unit_name, plant_name, actual_energy, daily_target_volume, and equipment_name in query if mentioned.\
            - Always use the column 'unit_name' associated with the unit in the generated query.\
            - Always use the column 'plant_name' in sql query generation whenever asked for plant or plant Names in user query. \
            - Always use the column 'actual_energy' associated with the energy consumption in the generated query.\
            - Always use the column 'daily_target_volume' associated with the production target, target volumne in the generated query.\
            - Always use the column 'unit_name' associated with the unit name [('Ammonia-Y1',), ('Urea-Y4',), ('Urea-Y2',), ('Mine-8',), ('Urea-Y1',), ('LNG-7',), ('Urea-Y3',), ('LNG-5',), ('Ammonia-Y3',), ('Ammonia-Y2',), ('Ammonia-Y4',), ('LNG-6',)] .\
            - Always use the column 'plant_name' associated with the plant name [('Y2',), ('Y3',), ('Y4',), ('Y1',)]. \
            - Always use the column 'equipment_name' associated with the Asset in the generated query.\
            - Whenever asked for plant or plant Names, return the institute names using column 'plant_name' associated with the 'plant_name' in the generated query.\
            - Likewise, Use appropriate aggregation functions (AVG, SUM). Use'AVG' when average word, 'SUM' when total or overall word is used. Else DO NOT use aggregate functions.\
            - Pay close attention to the filtering criteria mentioned in the question and incorporate them using the WHERE clause in your SQL query.\
            - If the question involves multiple conditions, use logical operators such as AND, OR to combine them effectively.\
            - When dealing with date or timestamp columns, use appropriate date functions (e.g., DATE_PART, EXTRACT) for extracting specific parts of the date or performing date arithmetic.\
            - If the question involves grouping of data (e.g., finding totals or averages for different categories), use the GROUP BY clause along with appropriate aggregate functions.\
            - Consider using aliases for tables and columns to improve readability of the query, especially in case of complex joins or subqueries.\
            - If necessary, use subqueries or common table expressions (CTEs) to break down the problem into smaller, more manageable parts. \
            - To determine the most energy-efficient or energy-inefficient plant, calculate the ratio by dividing the sum of actual production volume by the sum of actual energy used. Display the result and group along with plant namev only and unit if required. Do not apply the HAVING clause in the query.\
            - Incorporate filtering criteria using the WHERE clause.\
            - Use date functions for date or timestamp columns.\
 
        2. After executing the SQL query, interpret the results and provide a meaningful, human-readable answer to the user's question. \
            - For example, if the query returns a numeric value, explain what it represents. \
            - If the query returns multiple rows, summarize the key insights instead of listing all rows. \
            - If the query involves trends or comparisons, describe them clearly.
 
        3. If the provided context is insufficient, explain why the query cannot be generated or why the question cannot be answered.
 
        4. Always format the SQL query for readability before responding.
 
        5. Always respond with a valid, well-formed JSON object in the following format:
        {{
            "SQLQuery": "Generated SQL query here",
            "SQLResult": "Raw SQL query result here",
            "Answer": "Human-readable interpretation of the result here"
        }}
       
        6. If the user query is about creating graphs or plots, generate a valid SQL query to produce the data required for plotting and explain how the data can be visualized.
 
        7. If you do not know the answer, reply as follows: {{"answer": "I do not know."}}
 
        ===Response Format
        You are required to use the following format, each taking one line:
 
        Question: Question here
        SQLQuery: SQL Query to run
        SQLResult: Result of the SQLQuery
        Answer: A detailed, human-readable final answer here
        """

    # Define your prompt template for sql query
    sql_template = """
    You are a SQL expert. Given an input question, create a syntactically correct SQL query to run and return ONLY the generated Query and nothing else. \
    You can order the results by a relevant column to return the most interesting examples in the database. Remember NOT include backticks ```sql ``` before and after the created query. Unless otherwise specified, do not return more than \
    {top_k} rows. Here is the relevant table info: {table_info} \
    Finally, Use only tables names and Column names mentioned in:\n\n to create correct SQL Query and pay close attention on which column is in which table. if context contains more than one tables then create a query by performing JOIN operation for the tables.\

    ===Response Guidelines
    1. Follow these Instructions for creating syntactically correct SQL query: \
        - Be sure not to query for columns that do not exist in the tables and use alias only where required.\
        - Always use the column 'unit_name' associated with the unit in the generated query.\
        - Always use the column 'actual_energy' to calculate energy consumption for the input question.\
        - Always use the column 'daily_target_volume' associated with the production target, target volumne in the generated query.\
        - Always use the column 'plant_name' in sql query generation whenever asked for plant or plant Names in user query. \
        - Always use the column 'unit_name'  for the unit name values 'Ammonia-Y1', 'Urea-Y4', 'Urea-Y2', 'Mine-8', 'Urea-Y1', 'LNG-7', 'Urea-Y3', 'LNG-5', 'Ammonia-Y3', 'Ammonia-Y2', 'Ammonia-Y4', 'LNG-6'.\
        - Always use the column 'plant_name' associated with the plant name [('Y2',), ('Y3',), ('Y4',), ('Y1',)]. \
        - Always use the column 'equipment_name' associated with the Asset in the generated query.\
        - Whenever asked for plant or plant Names, return the institute names using column 'plant_name' associated with the 'plant_name' in the generated query.\
        - When the user requests plant names or all plant names along with unit names, ensure that the generated query displays both plant_name and unit_name columns.\
        - Likewise, when asked about the average (AVG function) or ratio, ensure the appropriate aggregation function is used.\
        - Pay close attention to the filtering criteria mentioned in the question and incorporate them using the WHERE clause in your SQL query.\
        - If the question involves multiple conditions, use logical operators such as AND, OR to combine them effectively.\
        - When dealing with date or timestamp columns, use appropriate date functions (e.g., DATE_PART, EXTRACT) for extracting specific parts of the date or performing date arithmetic.\
        - If the question involves grouping of data (e.g., finding totals or averages for different categories, finding value per unit_name or plant), use the GROUP BY clause along with appropriate aggregate functions.\
        - Consider using aliases for tables and columns to improve readability of the query, especially in case of complex joins or subqueries.\
        - If necessary, use subqueries or common table expressions (CTEs) to break down the problem into smaller, more manageable parts. \

    2. If the provided context is sufficient, please generate a valid query without any explanations for the question.
    3. If the provided context is insufficient, please explain why it can't be generated.
    4. Please use the most relevant table(s).
    5. you restrict 'sql_db_query' invoking with query  upto only one time. 
    6. you restrict 'sql_db_query_checker' invoking with query upto only one time.  
    7. Please format the query before responding.
    8. Please always respond with a valid well-formed JSON object with the following format.
    9. If user query is about creation of graphs and plots in that case please generate a valid sql query to produce the data which will be use in ploting . 
    10. If the question does not seem related to the database, just return 'I don't know' as the answer. 

    ===Response Format
    {{
        "query": "A generated SQL query when context is sufficient.",
        "explanation": "An explanation of failing to generate the query."
    }}

    """

    def define_prompt(self,template, context_text):
        # Create a ChatPromptTemplate
        sql_prompt = template.format(
        dialect=db.dialect,
        top_k=5,
        table_info=context_text)

        prompt = ChatPromptTemplate.from_messages(
                [
                    ("system", sql_prompt),
                    ("human", "{input}"),
                    MessagesPlaceholder(variable_name="agent_scratchpad"),
                ]
            )
        return prompt

In [30]:
from langchain.agents import create_sql_agent
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit


toolkit = SQLDatabaseToolkit(db=db, llm=llm_instance)

import numpy as np

In [31]:
def agent_trigger(question):
    # Define your prompt template
    prompt_templates = PromptTemplate()

    context = query_faiss(question)


    string_prompt = prompt_templates.define_prompt(prompt_templates.string_template, context)
    sql_prompt = prompt_templates.define_prompt(prompt_templates.sql_template, context)

    dbr_agent = create_sql_agent(
        llm=llm_instance,
        toolkit=toolkit,\
        prompt=string_prompt,
        agent_type= "openai-tools",
        handle_parsing_errors=True,
        verbose=True
        )

    sql_dbr_agent = create_sql_agent(
                llm=llm_instance,
                toolkit=toolkit,
                prompt=sql_prompt,
                agent_type="openai-tools",
                handle_parsing_errors=True,
                verbose=True,
                max_iterations=5
            )
    return (dbr_agent.invoke(question),sql_dbr_agent.invoke(question))

In [ ]:
question = "What iwas the most energy efficient plant in 2024?"
response  = agent_trigger(question)